# import

In [149]:
import pandas as pd

# 보통 현재 작업 폴더(CWD)가 src 이므로 부모(프로젝트 루트)를 sys.path에 추가
import sys, os
from pathlib import Path

ROOT = Path.cwd().parent
if (ROOT / "utils").exists():
    sys.path.insert(0, str(ROOT))
else:
    # 혹시 구조가 더 깊을 때 대비
    ROOT2 = ROOT.parent
    if (ROOT2 / "utils").exists():
        sys.path.insert(0, str(ROOT2))

from utils.angular_util import compute_joint_angles, compute_connected_unit_vectors, flatten_vectors, compute_face_hand_vectors

## 수집한 데이터 합치기

In [150]:
HAND_COUNT = 21 * 3
POSE_COUNT = 11 * 3

## 헤더 추가
🚨 주의!!!! 이미 헤더가 있는 경우 또 추가될 수 있음!

In [151]:
# for i in range(7, 8):
#     data = pd.read_csv(f"../../data/sign_data/sign_data_{i}.csv")
    # data.insert(0, 'label', [i] * len(data))
    # columns = [i for i in range(len(data.columns) - 1)]
    # columns.insert(0, 'label')
    # data.columns = columns
    # data.to_csv(f"../../data/sign_data/sign_data_{i}.csv", index=False)

In [152]:
# i = 5
# data = pd.read_csv(f"../../data/sign_data/sign_data_{i}.csv", index_col=False)
# data.loc[:, 'label'] = i
# data.to_csv(f"../../data/sign_data/sign_data_{i}.csv", index=False)

## 병합

In [153]:
merged_data = pd.DataFrame()

for i, label in enumerate([0, 1, 2, 3, 4, 5, 6, 14, 16, 24]):
    print(label)
    data = pd.read_csv(f"../../data/sign_data/sign_data_{label}.csv", index_col=False)
    data = data.drop(['label'], axis=1)
    data.loc[:, 'label'] = i
    data = data.sample(n = 500, replace=False)
    merged_data = pd.concat([merged_data, data], ignore_index=True)

df = merged_data

0
1
2
3
4
5
6
14
16
24


In [154]:
df

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,label
0,0,0,0,0,0,0,0,0,0,0,...,0.415956,0.480910,-0.345598,0.515833,0.553830,-0.561763,0.461848,0.553832,-0.562273,0
1,0,0,0,0,0,0,0,0,0,0,...,0.425718,0.468927,-0.291953,0.525497,0.547344,-0.440874,0.472502,0.543820,-0.450870,0
2,0,0,0,0,0,0,0,0,0,0,...,0.412183,0.414075,-0.250955,0.509104,0.499478,-0.436761,0.457860,0.496353,-0.445405,0
3,0,0,0,0,0,0,0,0,0,0,...,0.411173,0.436976,-0.258918,0.513541,0.508416,-0.523642,0.455887,0.510107,-0.517213,0
4,0,0,0,0,0,0,0,0,0,0,...,0.424455,0.442690,-0.291590,0.523954,0.510749,-0.535381,0.472519,0.511365,-0.535433,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0.373312,0.448539,-0.367303,0.499905,0.546677,-0.630056,0.434673,0.549051,-0.630718,9
4996,0,0,0,0,0,0,0,0,0,0,...,0.373814,0.489353,-0.267193,0.471683,0.568039,-0.523025,0.418739,0.578124,-0.509707,9
4997,0,0,0,0,0,0,0,0,0,0,...,0.383658,0.454852,-0.297806,0.485475,0.544983,-0.568188,0.425197,0.542943,-0.552346,9
4998,0,0,0,0,0,0,0,0,0,0,...,0.364269,0.427086,-0.373485,0.480384,0.520014,-0.676240,0.414366,0.523889,-0.666826,9


In [155]:
# merged_data.to_csv("../../data/merge_num_7_data.csv", index=False)

## 데이터 불러오기

In [156]:
# import pandas as pd
# df = pd.read_csv("../../data/merge_num_7_data.csv")

## 클래스 확인

In [157]:
df['label'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

## x, y 데이터

In [158]:
y = df['label']
x = df.drop(['label'], axis=1)

print(x.shape, y.shape)

(5000, 159) (5000,)


# 데이터 전처리

In [159]:
right_points_x = x.iloc[:, 63:63+63]
face_points_x = x.iloc[:, 63+63:]
nose_point_x = face_points_x.iloc[:, 0]
print(right_points_x.shape, face_points_x.shape, nose_point_x.shape)

(5000, 63) (5000, 33) (5000,)


In [160]:
angles_list = []
vector_list = []
face_hand_vector_list = []
for i in range(len(right_points_x)):
    hand_row = right_points_x.iloc[i].tolist()
    face_row = face_points_x.iloc[i].tolist()
    angles, _, _ = compute_joint_angles(hand_row)
    angles_list.append(angles)

    _, _, vector = compute_connected_unit_vectors(hand_row, return_flat=True)
    vector_list.append(vector)

    _, _, face_hand_vector = compute_face_hand_vectors(face_row, hand_row)
    face_hand_vector_list.append(face_hand_vector)

print(len(angles_list[0]), len(vector_list[0]))

angular_x = pd.DataFrame(angles_list, columns=None)
vector_x = pd.DataFrame(vector_list, columns=None)
face_hand_vector_x = pd.DataFrame(face_hand_vector_list, columns=None)

print(angular_x.shape, vector_x.shape, face_hand_vector_x.shape)

15 72
(5000, 15) (5000, 72) (5000, 54)


In [161]:
result_x = pd.concat([angular_x, vector_x, face_hand_vector_x], axis=1)
result_x.columns = range(len(result_x.columns))

print(result_x.shape)

(5000, 141)


In [162]:
# result_x.to_csv("../../data/angle_vector_num_7_data.csv", index=False)

## 데이터 분할하기

In [163]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(result_x, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

## 데이터 스케일링

In [164]:
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# x_train = mms.fit_transform(x_train)
# x_test = mms.transform(x_test)

## 모델 학습

In [165]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

model = XGBClassifier(n_estimators=200, max_depth=3, learning_rate=0.01)
#model = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=42)
#model = LogisticRegression(solver='liblinear', max_iter=100)
model.fit(x_train, y_train)

score = model.score(x_test, y_test)

print(f"Accuracy: {np.round(score*100, 2)}%")

Accuracy: 98.7%


In [166]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.98      0.99       100
           1       0.96      0.99      0.98       100
           2       0.98      0.97      0.97       100
           3       0.99      0.99      0.99       100
           4       1.00      1.00      1.00       100
           5       0.98      0.97      0.97       100
           6       1.00      0.99      0.99       100
           7       1.00      1.00      1.00       100
           8       1.00      1.00      1.00       100
           9       0.96      0.98      0.97       100

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000



## 모델 저장하기

In [167]:
import joblib

joblib.dump(model, "../../models/xgb_sample_angle_vector_model.pkl")

['../../models/xgb_sample_angle_vector_model.pkl']